In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval
import gurobipy as gp
from gurobipy import quicksum
from gurobipy import GRB
import csv
import matplotlib.pyplot as plt
import ast

In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
# df = pd.read_csv('onLastModelweUseR1.csv')
df = pd.read_csv('onLastModelweUseW033P035_3days.csv')
# df['resultsS2E2'][0]


In [4]:
df.head()


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,HOUSEID,HHFAMINC,STRTTIME,ENDTIME,TRVLCMIN,TRPMILES,DWELTIME,CENSUS_D,WHYTRP1S_GENERAL,WHYFROM_GENERAL,states,public_charge,work_charge,max_j,Cluster_new_features,LowerLine,UpperLine,income_classes,public_avail_share,work_avail_share,public_charge_assign,work_charge_assign,3d_locations,3d_vmt,3d_dwell,power,ePrice,resultsS3E2,resultsS3E8,resultsS5E2,resultsS5E8
0,0,0,0,30000094,30000,"[740, 1340, 1420, 1705]","[755, 1405, 1438, 1715]","[15, 25, 18, 10]","[8.05, 5.883, 5.898, 4.864]","[345.0, 15.0, 147.0, 411.0]",9,"['Work', 'Work', 'Work', 'Home']","['Home', 'Work', 'Work', 'Work', 'Home']","['""\'Home\'""', ' "" \'Work\'""', ' "" \'Work\'""',...",1,1,5,2,36996,187706,0,0.330093,0.035370,1,0,"['Home', 'Work', 'Work', 'Work', 'Home', 'Work...","[8.05, 5.883, 5.898, 4.864, 8.05, 5.883, 5.898...","[345.0, 15.0, 147.0, 411.0, 345.0, 15.0, 147.0...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]","[0.095, 0.33, 0.33, 0.33, 0.095, 0.33, 0.33, 0...","[1.0, 3.2366161616161633, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.7421717171717184, 0.0, 0.0, 0.0, 4.446...","[1.0, 0.0, 0.0, 0.0, 0.0, 2.226515151515153, 0...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.9792929292929298, ..."
1,1,1,1,30000095,62500,"[1000, 1140, 1230, 1415, 1600, 1700, 2000, 2330]","[1015, 1155, 1250, 1430, 1610, 1715, 2015, 2345]","[15, 15, 20, 15, 10, 15, 15, 15]","[1.926, 3.247, 3.896, 6.34, 1.226, 3.618, 3.64...","[75.0, 35.0, 85.0, 90.0, 50.0, 165.0, 195.0, 3...",3,"['Public', 'Public', 'Public', 'Public', 'Publ...","['Home', 'Public', 'Public', 'Public', 'Public...","['""\'Home\'""', ' "" \'Public\'""', ' "" \'Public\...",0,0,9,2,26319,108377,1,0.047394,0.007430,0,0,"['Home', 'Public', 'Public', 'Public', 'Public...","[1.926, 3.247, 3.896, 6.34, 1.226, 3.618, 3.64...","[75.0, 35.0, 85.0, 90.0, 50.0, 165.0, 195.0, 3...","[6, 50, 50, 50, 50, 50, 50, 50, 6, 50, 50, 50,...","[0.103, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0....",infeasible,infeasible,infeasible,infeasible
2,2,2,2,30000194,5000,"[930, 1230]","[1015, 1430]","[45, 120]","[8.312, 8.202]","[135.0, 582.0]",9,"['Public', 'Home']","['Home', 'Public', 'Home']","['""\'Home\'""', ' "" \'Public\'""', ' "" \'Home\'""']",0,0,3,1,37025,163205,0,0.330093,0.035370,0,0,"['Home', 'Public', 'Home', 'Public', 'Home', '...","[8.312, 8.202, 8.312, 8.202, 8.312, 8.202]","[135.0, 582.0, 135.0, 582.0, 135.0, 582.0]","[6, 50, 6, 50, 6, 50, 6]","[0.095, 0.35, 0.095, 0.35, 0.095, 0.35, 0.095]",infeasible,infeasible,"[1.0, 0.1529292929292929, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.45349090909090906, 0.0, 0.02..."
3,3,3,3,30000227,62500,"[845, 1045]","[910, 1110]","[25, 25]","[9.105, 9.054]","[95.0, 774.0]",9,"['Public', 'Home']","['Home', 'Public', 'Home']","['""\'Home\'""', ' "" \'Public\'""', ' "" \'Home\'""']",0,0,3,2,36996,187706,1,0.330093,0.035370,0,0,"['Home', 'Public', 'Home', 'Public', 'Home', '...","[9.105, 9.054, 9.105, 9.054, 9.105, 9.054]","[95.0, 774.0, 95.0, 774.0, 95.0, 774.0]","[6, 50, 6, 50, 6, 50, 6]","[0.095, 0.35, 0.095, 0.35, 0.095, 0.35, 0.095]",infeasible,infeasible,"[1.0, 0.0, 0.0, 0.31909090909090904, 0.0, 0.0,...","[1.0, 0.0, 0.0, 0.46826666666666666, 0.0, 0.04..."
4,4,4,4,30000309,87500,"[1400, 1445]","[1405, 1450]","[5, 5]","[1.688, 1.698]","[40.0, 570.0]",7,"['Public', 'Home']","['Home', 'Public', 'Home']","['""\'Home\'""', ' "" \'Public\'""', ' "" \'Home\'""']",1,1,3,2,26114,124754,1,0.036816,0.007882,0,0,"['Home', 'Public', 'Home', 'Public', 'Home', '...","[1.688, 1.698, 1.688, 1.698, 1.688, 1.698]","[40.0, 570.0, 40.0, 570.0, 40.0, 570.0]","[6, 50, 6, 50, 6, 50, 6]","[0.078, 0.35, 0.078, 0.35, 0.078, 0.35, 0.078]","[1.0, 0.0009575757575757651, 0.0, 0.0, 0.0, 0....","[1.0, 0.0, 0.0, 0.0, 0.0, 0.3645939393939394, ...","[-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.2433818181818182, ..."


In [25]:
# df['resultsS3E2'] = df['resultsS3E2'].str[1:-1].str.split(',').tolist()
# df['resultsS3E8'] = df['resultsS3E8'].str[1:-1].str.split(',').tolist()
# df['resultsS5E2'] = df['resultsS5E2'].str[1:-1].str.split(',').tolist()
# df['resultsS5E8'] = df['resultsS5E8'].str[1:-1].str.split(',').tolist()


In [26]:
# Convert entire column to a list
# df["resultsS5E2"] = df["resultsS5E2"].apply(ast.literal_eval)

# df['resultsS3E2'] = df['resultsS3E2'].apply(lambda x: literal_eval(x))
# df['resultsS3E8'] = df['resultsS3E8'].apply(lambda x: literal_eval(x))
# df['resultsS5E2'] = df['resultsS5E2'].apply(lambda x: literal_eval(x))
# df['resultsS5E8'] = df['resultsS5E8'].apply(lambda x: literal_eval(x))

In [5]:
def deploy_or_not(x):
#     if type(x)==list:
#         if x[-1] > 0:
#             return 1
        return x[0]
#     else:
#         return 0

In [6]:
df['Cluster_new_features'] = df['Cluster_new_features'].astype('int')

In [7]:
cluster0 = df[df['Cluster_new_features'] == 0]
cluster1 = df[df['Cluster_new_features'] == 1]
cluster2 = df[df['Cluster_new_features'] == 2]

In [8]:
cluster0S3E2 = cluster0[cluster0['resultsS3E2']!='infeasible']
cluster0S3E2.loc[:,'resultsS3E2'] = cluster0S3E2.loc[:,'resultsS3E2'].apply(lambda x: literal_eval(x))
cluster0S3E2['deployS3E2']=cluster0S3E2['resultsS3E2'].agg(deploy_or_not)
# cluster0S3E2.head()
sharelistS3E2_C1 = np.mean(cluster0S3E2['deployS3E2'])
    
    
cluster0S3E8 = cluster0[cluster0['resultsS3E8']!='infeasible']
cluster0S3E8.loc[:,'resultsS3E8'] = cluster0S3E8.loc[:,'resultsS3E8'].apply(lambda x: literal_eval(x))
cluster0S3E8['deployS3E8']=cluster0S3E8['resultsS3E8'].agg(deploy_or_not)
# cluster0S3E2.head()
sharelistS3E8_C1 = np.mean(cluster0S3E8['deployS3E8'])
    

cluster0S5E2 = cluster0[cluster0['resultsS5E2']!='infeasible']
cluster0S5E2.loc[:,'resultsS5E2'] = cluster0S5E2.loc[:,'resultsS5E2'].apply(lambda x: literal_eval(x))
cluster0S5E2['deployS5E2']=cluster0S5E2['resultsS5E2'].agg(deploy_or_not)
# cluster0S3E2.head()
sharelistS5E2_C1 = np.mean(cluster0S5E2['deployS5E2'])
    

cluster0S5E8 = cluster0[cluster0['resultsS5E8']!='infeasible']
cluster0S5E8.loc[:,'resultsS5E8'] = cluster0S5E8.loc[:,'resultsS5E8'].apply(lambda x: literal_eval(x))
cluster0S5E8['deployS5E8']=cluster0S5E8['resultsS5E8'].agg(deploy_or_not)
# cluster0S3E2.head()
sharelistS5E8_C1 = np.mean(cluster0S5E8['deployS5E8'])

    

/Users/cheng_xi/opt/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/var/folders/6_/zl7m76ns3ld_pgnd6qth25mc0000gn/T/ipykernel_39237/195422196.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster0S3E2['deployS3E2']=cluster0S3E2['resultsS3E2'].agg(deploy_or_not)
/var/folders/6_/zl7m76ns3ld_pgnd6qth25mc0000gn/T/ipykernel_39237/195422196.py:10: SettingWithCopyWarning: 
A value is trying to

In [9]:
cluster1S3E2 = cluster1[cluster1['resultsS3E2']!='infeasible']
cluster1S3E2_copy = cluster1S3E2.reset_index()
for i in range(len(cluster1S3E2)):
    if type(cluster1S3E2_copy.loc[i,'resultsS3E2']) == list:
        cluster1S3E2_copy.loc[i,'deployS3E2']=cluster1S3E2_copy.loc[i,'resultsS3E2'][0]
    else:
        cluster1S3E2_copy.loc[i,'deployS3E2']=literal_eval(cluster1S3E2_copy.loc[i,'resultsS3E2'])[0]
sharelistS3E2_C2 = np.mean(cluster1S3E2_copy['deployS3E2'])

    
cluster1S3E8 = cluster1[cluster1['resultsS3E8']!='infeasible']
cluster1S3E8_copy = cluster1S3E8.reset_index()
for i in range(len(cluster1S3E8)):
    if type(cluster1S3E8_copy.loc[i,'resultsS3E8']) == list:
        cluster1S3E8_copy.loc[i,'deployS3E8']=cluster1S3E8_copy.loc[i,'resultsS3E8'][0]
    else:
        cluster1S3E8_copy.loc[i,'deployS3E8']=literal_eval(cluster1S3E8_copy.loc[i,'resultsS3E8'])[0]
sharelistS3E8_C2 = np.mean(cluster1S3E8_copy['deployS3E8'])


cluster1S5E2 = cluster1[cluster1['resultsS5E2']!='infeasible']
cluster1S5E2_copy = cluster1S5E2.reset_index()
for i in range(len(cluster1S5E2)):
    if type(cluster1S5E2_copy.loc[i,'resultsS5E2']) == list:
        cluster1S5E2_copy.loc[i,'deployS5E2']=cluster1S5E2_copy.loc[i,'resultsS5E2'][0]
    else:
        cluster1S5E2_copy.loc[i,'deployS5E2']=literal_eval(cluster1S5E2_copy.loc[i,'resultsS5E2'])[0]
sharelistS5E2_C2 = np.mean(cluster1S5E2_copy['deployS5E2'])
    
    
cluster1S5E8 = cluster1[cluster1['resultsS5E8']!='infeasible']
cluster1S5E8_copy = cluster1S5E8.reset_index()
for i in range(len(cluster1S5E8)):
    if type(cluster1S5E8_copy.loc[i,'resultsS5E8']) == list:
        cluster1S5E8_copy.loc[i,'deployS5E8']=cluster1S5E8_copy.loc[i,'resultsS5E8'][0]
    else:
        cluster1S5E8_copy.loc[i,'deployS5E8']=literal_eval(cluster1S5E8_copy.loc[i,'resultsS5E8'])[0]
sharelistS5E8_C2 = np.mean(cluster1S5E8_copy['deployS5E8'])



In [10]:
cluster2S3E2 = cluster2[cluster2['resultsS3E2']!='infeasible']
cluster2S3E2_copy = cluster2S3E2.reset_index()
for i in range(len(cluster2S3E2)):
    if type(cluster2S3E2_copy.loc[i,'resultsS3E2']) == list:
        cluster2S3E2_copy.loc[i,'deployS3E2']=cluster2S3E2_copy.loc[i,'resultsS3E2'][0]
    else:
        cluster2S3E2_copy.loc[i,'deployS3E2']=literal_eval(cluster2S3E2_copy.loc[i,'resultsS3E2'])[0]
sharelistS3E2_C3 = np.mean(cluster2S3E2_copy['deployS3E2'])

    
cluster2S3E8 = cluster2[cluster2['resultsS3E8']!='infeasible']
cluster2S3E8_copy = cluster2S3E8.reset_index()
for i in range(len(cluster2S3E8)):
    if type(cluster2S3E8_copy.loc[i,'resultsS3E8']) == list:
        cluster2S3E8_copy.loc[i,'deployS3E8']=cluster2S3E8_copy.loc[i,'resultsS3E8'][0]
    else:
        cluster2S3E8_copy.loc[i,'deployS3E8']=literal_eval(cluster2S3E8_copy.loc[i,'resultsS3E8'])[0]
sharelistS3E8_C3 = np.mean(cluster2S3E8_copy['deployS3E8'])


cluster2S5E2 = cluster2[cluster2['resultsS5E2']!='infeasible']
cluster2S5E2_copy = cluster2S5E2.reset_index()
for i in range(len(cluster2S5E2)):
    if type(cluster2S5E2_copy.loc[i,'resultsS5E2']) == list:
        cluster2S5E2_copy.loc[i,'deployS5E2']=cluster2S5E2_copy.loc[i,'resultsS5E2'][0]
    else:
        cluster2S5E2_copy.loc[i,'deployS5E2']=literal_eval(cluster2S5E2_copy.loc[i,'resultsS5E2'])[0]
sharelistS5E2_C3 = np.mean(cluster2S5E2_copy['deployS5E2'])
    
    
cluster2S5E8 = cluster2[cluster2['resultsS5E8']!='infeasible']
cluster2S5E8_copy = cluster2S5E8.reset_index()
for i in range(len(cluster2S5E8)):
    if type(cluster2S5E8_copy.loc[i,'resultsS5E8']) == list:
        cluster2S5E8_copy.loc[i,'deployS5E8']=cluster2S5E8_copy.loc[i,'resultsS5E8'][0]
    else:
        cluster2S5E8_copy.loc[i,'deployS5E8']=literal_eval(cluster2S5E8_copy.loc[i,'resultsS5E8'])[0]
sharelistS5E8_C3 = np.mean(cluster2S5E8_copy['deployS5E8'])



In [11]:
MUD = pd.DataFrame()
  
# print(MudShare)
  
# append columns to an empty DataFrame
MUD['ANALYSIS'] = ['MUD share', 'TOTAL energy use', 'MUD energy use']

MUD.loc[0,'Cluster 1 (I30E20)'] = sharelistS3E2_C1
MUD.loc[0,'Cluster 2 (I30E20)'] = sharelistS3E2_C2
MUD.loc[0,'Cluster 3 (I30E20)'] = sharelistS3E2_C3

MUD.loc[0,'Cluster 1 (I30E80)'] = sharelistS3E8_C1
MUD.loc[0,'Cluster 2 (I30E80)'] = sharelistS3E8_C2
MUD.loc[0,'Cluster 3 (I30E80)'] = sharelistS3E8_C3

MUD.loc[0,'Cluster 1 (I50E20)'] = sharelistS5E2_C1
MUD.loc[0,'Cluster 2 (I50E20)'] = sharelistS5E2_C2
MUD.loc[0,'Cluster 3 (I50E20)'] = sharelistS5E2_C3

MUD.loc[0,'Cluster 1 (I50E80)'] = sharelistS5E8_C1
MUD.loc[0,'Cluster 2 (I50E80)'] = sharelistS5E8_C2
MUD.loc[0,'Cluster 3 (I50E80)'] = sharelistS5E8_C3

In [13]:
MUD

,ANALYSIS,Cluster 1 (I30E20),Cluster 2 (I30E20),Cluster 3 (I30E20),Cluster 1 (I30E80),Cluster 2 (I30E80),Cluster 3 (I30E80),Cluster 1 (I50E20),Cluster 2 (I50E20),Cluster 3 (I50E20),Cluster 1 (I50E80),Cluster 2 (I50E80),Cluster 3 (I50E80)
0,MUD share,0.766694,0.696864,0.832773,0.989085,1.0,0.945201,0.435525,0.383117,0.59633,0.991717,1.0,0.959055
1,TOTAL energy use,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MUD energy use,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.head()

In [14]:
income0 = df[df['income_classes'] == 0]
income1 = df[df['income_classes'] == 1]
income2 = df[df['income_classes'] == 2]

In [15]:
income0S3E2 = income0[income0['resultsS3E2']!='infeasible']
income0S3E2_copy = income0S3E2.reset_index()
for i in range(len(income0S3E2)):
    if type(income0S3E2_copy.loc[i,'resultsS3E2']) == list:
        income0S3E2_copy.loc[i,'deployS3E2']=income0S3E2_copy.loc[i,'resultsS3E2'][0]
    else:
        income0S3E2_copy.loc[i,'deployS3E2']=literal_eval(income0S3E2_copy.loc[i,'resultsS3E2'])[0]
sharelistS3E2_I0 = np.mean(income0S3E2_copy['deployS3E2'])

    
income0S3E8 = income0[income0['resultsS3E8']!='infeasible']
income0S3E8_copy = income0S3E8.reset_index()
for i in range(len(income0S3E8)):
    if type(income0S3E8_copy.loc[i,'resultsS3E8']) == list:
        income0S3E8_copy.loc[i,'deployS3E8']=income0S3E8_copy.loc[i,'resultsS3E8'][0]
    else:
        income0S3E8_copy.loc[i,'deployS3E8']=literal_eval(income0S3E8_copy.loc[i,'resultsS3E8'])[0]
sharelistS3E8_I0 = np.mean(income0S3E8_copy['deployS3E8'])


income0S5E2 = income0[income0['resultsS5E2']!='infeasible']
income0S5E2_copy = income0S5E2.reset_index()
for i in range(len(income0S5E2)):
    if type(income0S5E2_copy.loc[i,'resultsS5E2']) == list:
        income0S5E2_copy.loc[i,'deployS5E2']=income0S5E2_copy.loc[i,'resultsS5E2'][0]
    else:
        income0S5E2_copy.loc[i,'deployS5E2']=literal_eval(income0S5E2_copy.loc[i,'resultsS5E2'])[0]
sharelistS5E2_I0 = np.mean(income0S5E2_copy['deployS5E2'])
    
    
income0S5E8 = income0[income0['resultsS5E8']!='infeasible']
income0S5E8_copy = income0S5E8.reset_index()
for i in range(len(income0S5E8)):
    if type(income0S5E8_copy.loc[i,'resultsS5E8']) == list:
        income0S5E8_copy.loc[i,'deployS5E8']=income0S5E8_copy.loc[i,'resultsS5E8'][0]
    else:
        income0S5E8_copy.loc[i,'deployS5E8']=literal_eval(income0S5E8_copy.loc[i,'resultsS5E8'])[0]
sharelistS5E8_I0 = np.mean(income0S5E8_copy['deployS5E8'])




In [16]:
income1S3E2 = income1[income1['resultsS3E2']!='infeasible']
income1S3E2_copy = income1S3E2.reset_index()
for i in range(len(income1S3E2)):
    if type(income1S3E2_copy.loc[i,'resultsS3E2']) == list:
        income1S3E2_copy.loc[i,'deployS3E2']=income1S3E2_copy.loc[i,'resultsS3E2'][0]
    else:
        income1S3E2_copy.loc[i,'deployS3E2']=literal_eval(income1S3E2_copy.loc[i,'resultsS3E2'])[0]
sharelistS3E2_I1 = np.mean(income1S3E2_copy['deployS3E2'])

    
income1S3E8 = income1[income1['resultsS3E8']!='infeasible']
income1S3E8_copy = income1S3E8.reset_index()
for i in range(len(income1S3E8)):
    if type(income1S3E8_copy.loc[i,'resultsS3E8']) == list:
        income1S3E8_copy.loc[i,'deployS3E8']=income1S3E8_copy.loc[i,'resultsS3E8'][0]
    else:
        income1S3E8_copy.loc[i,'deployS3E8']=literal_eval(income1S3E8_copy.loc[i,'resultsS3E8'])[0]
sharelistS3E8_I1 = np.mean(income1S3E8_copy['deployS3E8'])


income1S5E2 = income1[income1['resultsS5E2']!='infeasible']
income1S5E2_copy = income1S5E2.reset_index()
for i in range(len(income1S5E2)):
    if type(income1S5E2_copy.loc[i,'resultsS5E2']) == list:
        income1S5E2_copy.loc[i,'deployS5E2']=income1S5E2_copy.loc[i,'resultsS5E2'][0]
    else:
        income1S5E2_copy.loc[i,'deployS5E2']=literal_eval(income1S5E2_copy.loc[i,'resultsS5E2'])[0]
sharelistS5E2_I1 = np.mean(income1S5E2_copy['deployS5E2'])
    
    
income1S5E8 = income1[income1['resultsS5E8']!='infeasible']
income1S5E8_copy = income1S5E8.reset_index()
for i in range(len(income1S5E8)):
    if type(income1S5E8_copy.loc[i,'resultsS5E8']) == list:
        income1S5E8_copy.loc[i,'deployS5E8']=income1S5E8_copy.loc[i,'resultsS5E8'][0]
    else:
        income1S5E8_copy.loc[i,'deployS5E8']=literal_eval(income1S5E8_copy.loc[i,'resultsS5E8'])[0]
sharelistS5E8_I1 = np.mean(income1S5E8_copy['deployS5E8'])




In [17]:
income2S3E2 = income2[income2['resultsS3E2']!='infeasible']
income2S3E2_copy = income2S3E2.reset_index()
for i in range(len(income2S3E2)):
    if type(income2S3E2_copy.loc[i,'resultsS3E2']) == list:
        income2S3E2_copy.loc[i,'deployS3E2']=income2S3E2_copy.loc[i,'resultsS3E2'][0]
    else:
        income2S3E2_copy.loc[i,'deployS3E2']=literal_eval(income2S3E2_copy.loc[i,'resultsS3E2'])[0]
sharelistS3E2_I2 = np.mean(income2S3E2_copy['deployS3E2'])


    
income2S3E8 = income2[income2['resultsS3E8']!='infeasible']
income2S3E8_copy = income2S3E8.reset_index()
for i in range(len(income2S3E8)):
    if type(income2S3E8_copy.loc[i,'resultsS3E8']) == list:
        income2S3E8_copy.loc[i,'deployS3E8']=income2S3E8_copy.loc[i,'resultsS3E8'][0]
    else:
        income2S3E8_copy.loc[i,'deployS3E8']=literal_eval(income2S3E8_copy.loc[i,'resultsS3E8'])[0]
sharelistS3E8_I2 = np.mean(income2S3E8_copy['deployS3E8'])



income2S5E2 = income2[income2['resultsS5E2']!='infeasible']
income2S5E2_copy = income2S5E2.reset_index()
for i in range(len(income2S5E2)):
    if type(income2S5E2_copy.loc[i,'resultsS5E2']) == list:
        income2S5E2_copy.loc[i,'deployS5E2']=income2S5E2_copy.loc[i,'resultsS5E2'][0]
    else:
        income2S5E2_copy.loc[i,'deployS5E2']=literal_eval(income2S5E2_copy.loc[i,'resultsS5E2'])[0]
sharelistS5E2_I2 = np.mean(income2S5E2_copy['deployS5E2'])

    
    
income2S5E8 = income2[income2['resultsS5E8']!='infeasible']
income2S5E8_copy = income2S5E8.reset_index()
for i in range(len(income2S5E8)):
    if type(income2S5E8_copy.loc[i,'resultsS5E8']) == list:
        income2S5E8_copy.loc[i,'deployS5E8']=income2S5E8_copy.loc[i,'resultsS5E8'][0]
    else:
        income2S5E8_copy.loc[i,'deployS5E8']=literal_eval(income2S5E8_copy.loc[i,'resultsS5E8'])[0]
sharelistS5E8_I2 = np.mean(income2S5E8_copy['deployS5E8'])




In [18]:

MUD.loc[0,'Income 1 (I30E20)'] = sharelistS3E2_I0
MUD.loc[0,'Income 2 (I30E20)'] = sharelistS3E2_I1
MUD.loc[0,'Income 3 (I30E20)'] = sharelistS3E2_I2

MUD.loc[0,'Income 1 (I30E80)'] = sharelistS3E8_I0
MUD.loc[0,'Income 2 (I30E80)'] = sharelistS3E8_I1
MUD.loc[0,'Income 3 (I30E80)'] = sharelistS3E8_I2

MUD.loc[0,'Income 1 (I50E20)'] = sharelistS5E2_I0
MUD.loc[0,'Income 2 (I50E20)'] = sharelistS5E2_I1
MUD.loc[0,'Income 3 (I50E20)'] = sharelistS5E2_I2

MUD.loc[0,'Income 1 (I50E80)'] = sharelistS5E8_I0
MUD.loc[0,'Income 2 (I50E80)'] = sharelistS5E8_I1
MUD.loc[0,'Income 3 (I50E80)'] = sharelistS5E8_I2

In [19]:
MUD

,ANALYSIS,Cluster 1 (I30E20),Cluster 2 (I30E20),Cluster 3 (I30E20),Cluster 1 (I30E80),Cluster 2 (I30E80),Cluster 3 (I30E80),Cluster 1 (I50E20),Cluster 2 (I50E20),Cluster 3 (I50E20),Cluster 1 (I50E80),Cluster 2 (I50E80),Cluster 3 (I50E80),Income 1 (I30E20),Income 2 (I30E20),Income 3 (I30E20),Income 1 (I30E80),Income 2 (I30E80),Income 3 (I30E80),Income 1 (I50E20),Income 2 (I50E20),Income 3 (I50E20),Income 1 (I50E80),Income 2 (I50E80),Income 3 (I50E80)
0,MUD share,0.766694,0.696864,0.832773,0.989085,1.0,0.945201,0.435525,0.383117,0.59633,0.991717,1.0,0.959055,0.757468,0.832507,0.928144,0.968459,0.954586,0.962963,0.442889,0.587696,0.734783,0.976802,0.966474,0.964286
1,TOTAL energy use,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MUD energy use,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# import xlsxwriter
# # Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pd.ExcelWriter('ShareEnergySummaryDiffLevel.xlsx', engine='xlsxwriter')

# # Write each dataframe to a different worksheet.
# MudShare.to_excel(writer, sheet_name='MUDShare_Cluster')
# MudShareIncome.to_excel(writer, sheet_name='MUDShare_Income')
# # df3.to_excel(writer, sheet_name='Sheet3')

In [18]:
MudShare.to_excel("ShareCluster_MUDLEVEL2_W033P035_3days.xlsx",
...              sheet_name='MUDShare_Cluster')  
MudShareIncome.to_excel("ShareIncome_MUDLEVEL2_W033P035_3days.xlsx",
...              sheet_name='MUDShare_Income')  

energy use

In [20]:
# df.loc[:,'WHYFROM_GENERAL'] = df.loc[:,'WHYFROM_GENERAL'].apply(lambda x: literal_eval(x))
# df.loc[0,'WHYFROM_GENERAL'][0]
df.loc[:,'3d_locations'] = df.loc[:,'3d_locations'].apply(lambda x: literal_eval(x))

In [39]:
literal_eval(df.loc[0,'resultsS3E2'])


[1.0,
 3.2366161616161633,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 7.827939685345342e-17]

In [38]:
len(literal_eval(df.loc[0,'WHYFROM_GENERAL']))

5

In [40]:
len(df.loc[0,'resultsS3E2'])

98

In [ ]:
energy = 0
for i in range(len(df.loc[0,'resultsS3E8'])-1):
    if df.loc[0,'WHYFROM_GENERAL'][i] == 'Home':
        energy = energy+P[0]*df.loc[0,'resultsS3E8'][i+1]
    elif df.loc[0,'WHYFROM_GENERAL'][i] == 'Work':
        energy = energy+P[1]*df.loc[0,'resultsS3E8'][i+1]
    elif df.loc[0,'WHYFROM_GENERAL'][i] == 'Public':
        energy = energy+P[2]*df.loc[0,'resultsS3E8'][i+1]
        
energy

In [64]:
#energy power:
P = [6, 0, 0]

In [ ]:
def energy_compute(x1,x2):
    energy = 0
    if x1 == 'infeasible':
        return 'infeasible'
    else:
        for i in range(len(x1)-1):
            if x2[i] == 'Home':
                energy = energy+P[0]*x1[i+1]
            elif x2[i] == 'Work':
                energy = energy+P[1]*x1[i+1]
            elif x2[i] == 'Public':
                energy = energy+P[2]*x1[i+1]

    return energy

In [65]:
energyListS3E2 = []
for i in range(len(df)): 
    if df.loc[i,'resultsS3E2'] == 'infeasible':
        energy = 'infeasible'
    else:
        energy = 0
        if type(df.loc[i,'resultsS3E2']) == list:
            for j in range(len(df.loc[i,'WHYFROM_GENERAL']),2*len(df.loc[i,'WHYFROM_GENERAL'])-1):
                if df.loc[i,'3d_locations'][j] == 'Home':
                    energy = energy + P[0]*df.loc[i,'resultsS3E2'][j+1]
                elif df.loc[i,'3d_locations'][j] == 'Work':
                    energy = energy + P[1]*df.loc[i,'resultsS3E2'][j+1]
                elif df.loc[i,'3d_locations'][j] == 'Public':
                    energy = energy + P[2]*df.loc[i,'resultsS3E2'][j+1]
        else:
            for j in range(len(literal_eval(df.loc[i,'WHYFROM_GENERAL'])),2*len(literal_eval(df.loc[i,'WHYFROM_GENERAL']))-1):
                if df.loc[i,'3d_locations'][j] == 'Home':
                    energy = energy + P[0]*literal_eval(df.loc[i,'resultsS3E2'])[j+1]
                elif df.loc[i,'3d_locations'][j] == 'Work':
                    energy = energy + P[1]*literal_eval(df.loc[i,'resultsS3E2'])[j+1]
                elif df.loc[i,'3d_locations'][j] == 'Public':
                    energy = energy + P[2]*literal_eval(df.loc[i,'resultsS3E2'])[j+1]
        
                    
    energyListS3E2.append(energy)
    


In [66]:
energyListS3E8 = []
for i in range(len(df)): 
    if df.loc[i,'resultsS3E8'] == 'infeasible':
        energy = 'infeasible'
    else:
        energy = 0
        if type(df.loc[i,'resultsS3E8']) == list:
            for j in range(len(df.loc[i,'WHYFROM_GENERAL']),2*len(df.loc[i,'WHYFROM_GENERAL'])-1):
                if df.loc[i,'3d_locations'][j] == 'Home':
                    energy = energy + P[0]*df.loc[i,'resultsS3E8'][j+1]
                elif df.loc[i,'3d_locations'][j] == 'Work':
                    energy = energy + P[1]*df.loc[i,'resultsS3E8'][j+1]
                elif df.loc[i,'3d_locations'][j] == 'Public':
                    energy = energy + P[2]*df.loc[i,'resultsS3E8'][j+1]
        else:
            for j in range(len(literal_eval(df.loc[i,'WHYFROM_GENERAL'])),2*len(literal_eval(df.loc[i,'WHYFROM_GENERAL']))-1):
                if df.loc[i,'3d_locations'][j] == 'Home':
                    energy = energy + P[0]*literal_eval(df.loc[i,'resultsS3E8'])[j+1]
                elif df.loc[i,'3d_locations'][j] == 'Work':
                    energy = energy + P[1]*literal_eval(df.loc[i,'resultsS3E8'])[j+1]
                elif df.loc[i,'3d_locations'][j] == 'Public':
                    energy = energy + P[2]*literal_eval(df.loc[i,'resultsS3E8'])[j+1]
        
                    
    energyListS3E8.append(energy)
    


In [67]:
energyListS5E2 = []
for i in range(len(df)): 
    if df.loc[i,'resultsS5E2'] == 'infeasible':
        energy = 'infeasible'
    else:
        energy = 0
        if type(df.loc[i,'resultsS5E2']) == list:
            for j in range(len(df.loc[i,'WHYFROM_GENERAL']),2*len(df.loc[i,'WHYFROM_GENERAL'])-1):
                if df.loc[i,'3d_locations'][j] == 'Home':
                    energy = energy + P[0]*df.loc[i,'resultsS5E2'][j+1]
                elif df.loc[i,'3d_locations'][j] == 'Work':
                    energy = energy + P[1]*df.loc[i,'resultsS5E2'][j+1]
                elif df.loc[i,'3d_locations'][j] == 'Public':
                    energy = energy + P[2]*df.loc[i,'resultsS5E2'][j+1]
        else:
            for j in range(len(literal_eval(df.loc[i,'WHYFROM_GENERAL'])),2*len(literal_eval(df.loc[i,'WHYFROM_GENERAL']))-1):
                if df.loc[i,'3d_locations'][j] == 'Home':
                    energy = energy + P[0]*literal_eval(df.loc[i,'resultsS5E2'])[j+1]
                elif df.loc[i,'3d_locations'][j] == 'Work':
                    energy = energy + P[1]*literal_eval(df.loc[i,'resultsS5E2'])[j+1]
                elif df.loc[i,'3d_locations'][j] == 'Public':
                    energy = energy + P[2]*literal_eval(df.loc[i,'resultsS5E2'])[j+1]
        
                    
    energyListS5E2.append(energy)
    



In [68]:
energyListS5E8 = []
for i in range(len(df)): 
    if df.loc[i,'resultsS5E8'] == 'infeasible':
        energy = 'infeasible'
    else:
        energy = 0
        if type(df.loc[i,'resultsS5E8']) == list:
            for j in range(len(df.loc[i,'WHYFROM_GENERAL']),2*len(df.loc[i,'WHYFROM_GENERAL'])-1):
                if df.loc[i,'3d_locations'][j] == 'Home':
                    energy = energy + P[0]*df.loc[i,'resultsS5E8'][j+1]
                elif df.loc[i,'3d_locations'][j] == 'Work':
                    energy = energy + P[1]*df.loc[i,'resultsS5E8'][j+1]
                elif df.loc[i,'3d_locations'][j] == 'Public':
                    energy = energy + P[2]*df.loc[i,'resultsS5E8'][j+1]
        else:
            for j in range(len(literal_eval(df.loc[i,'WHYFROM_GENERAL'])),2*len(literal_eval(df.loc[i,'WHYFROM_GENERAL']))-1):
                if df.loc[i,'3d_locations'][j] == 'Home':
                    energy = energy + P[0]*literal_eval(df.loc[i,'resultsS5E8'])[j+1]
                elif df.loc[i,'3d_locations'][j] == 'Work':
                    energy = energy + P[1]*literal_eval(df.loc[i,'resultsS5E8'])[j+1]
                elif df.loc[i,'3d_locations'][j] == 'Public':
                    energy = energy + P[2]*literal_eval(df.loc[i,'resultsS5E8'])[j+1]
        
                    
    energyListS5E8.append(energy)
    


In [69]:
df.loc[0,'3d_locations'][0]

'Home'

In [70]:
df['energyS3E2'] = energyListS3E2
df['energyS3E8'] = energyListS3E8
df['energyS5E2'] = energyListS5E2
df['energyS5E8'] = energyListS5E8
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,HOUSEID,HHFAMINC,STRTTIME,ENDTIME,TRVLCMIN,TRPMILES,DWELTIME,CENSUS_D,WHYTRP1S_GENERAL,WHYFROM_GENERAL,states,public_charge,work_charge,max_j,Cluster_new_features,LowerLine,UpperLine,income_classes,public_avail_share,work_avail_share,public_charge_assign,work_charge_assign,3d_locations,3d_vmt,3d_dwell,power,ePrice,resultsS3E2,resultsS3E8,resultsS5E2,resultsS5E8,energyS3E2,energyS3E8,energyS5E2,energyS5E8
0,0,0,0,30000094,30000,"[740, 1340, 1420, 1705]","[755, 1405, 1438, 1715]","[15, 25, 18, 10]","[8.05, 5.883, 5.898, 4.864]","[345.0, 15.0, 147.0, 411.0]",9,"['Work', 'Work', 'Work', 'Home']","['Home', 'Work', 'Work', 'Work', 'Home']","['""\'Home\'""', ' "" \'Work\'""', ' "" \'Work\'""',...",1,1,5,2,36996,187706,0,0.330093,0.035370,1,0,"[Home, Work, Work, Work, Home, Work, Work, Wor...","[8.05, 5.883, 5.898, 4.864, 8.05, 5.883, 5.898...","[345.0, 15.0, 147.0, 411.0, 345.0, 15.0, 147.0...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]","[0.095, 0.33, 0.33, 0.33, 0.095, 0.33, 0.33, 0...","[1.0, 3.2366161616161633, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.7421717171717184, 0.0, 0.0, 0.0, 4.446...","[1.0, 0.0, 0.0, 0.0, 0.0, 2.226515151515153, 0...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.9792929292929298, ...",0.0,6.466667,0.0,25.665152
1,1,1,1,30000095,62500,"[1000, 1140, 1230, 1415, 1600, 1700, 2000, 2330]","[1015, 1155, 1250, 1430, 1610, 1715, 2015, 2345]","[15, 15, 20, 15, 10, 15, 15, 15]","[1.926, 3.247, 3.896, 6.34, 1.226, 3.618, 3.64...","[75.0, 35.0, 85.0, 90.0, 50.0, 165.0, 195.0, 3...",3,"['Public', 'Public', 'Public', 'Public', 'Publ...","['Home', 'Public', 'Public', 'Public', 'Public...","['""\'Home\'""', ' "" \'Public\'""', ' "" \'Public\...",0,0,9,2,26319,108377,1,0.047394,0.007430,0,0,"[Home, Public, Public, Public, Public, Public,...","[1.926, 3.247, 3.896, 6.34, 1.226, 3.618, 3.64...","[75.0, 35.0, 85.0, 90.0, 50.0, 165.0, 195.0, 3...","[6, 50, 50, 50, 50, 50, 50, 50, 6, 50, 50, 50,...","[0.103, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0....",infeasible,infeasible,infeasible,infeasible,infeasible,infeasible,infeasible,infeasible
2,2,2,2,30000194,5000,"[930, 1230]","[1015, 1430]","[45, 120]","[8.312, 8.202]","[135.0, 582.0]",9,"['Public', 'Home']","['Home', 'Public', 'Home']","['""\'Home\'""', ' "" \'Public\'""', ' "" \'Home\'""']",0,0,3,1,37025,163205,0,0.330093,0.035370,0,0,"[Home, Public, Home, Public, Home, Public, Home]","[8.312, 8.202, 8.312, 8.202, 8.312, 8.202]","[135.0, 582.0, 135.0, 582.0, 135.0, 582.0]","[6, 50, 6, 50, 6, 50, 6]","[0.095, 0.35, 0.095, 0.35, 0.095, 0.35, 0.095]",infeasible,infeasible,"[1.0, 0.1529292929292929, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.45349090909090906, 0.0, 0.02...",infeasible,infeasible,0.0,0.171491
3,3,3,3,30000227,62500,"[845, 1045]","[910, 1110]","[25, 25]","[9.105, 9.054]","[95.0, 774.0]",9,"['Public', 'Home']","['Home', 'Public', 'Home']","['""\'Home\'""', ' "" \'Public\'""', ' "" \'Home\'""']",0,0,3,2,36996,187706,1,0.330093,0.035370,0,0,"[Home, Public, Home, Public, Home, Public, Home]","[9.105, 9.054, 9.105, 9.054, 9.105, 9.054]","[95.0, 774.0, 95.0, 774.0, 95.0, 774.0]","[6, 50, 6, 50, 6, 50, 6]","[0.095, 0.35, 0.095, 0.35, 0.095, 0.35, 0.095]",infeasible,infeasible,"[1.0, 0.0, 0.0, 0.31909090909090904, 0.0, 0.0,...","[1.0, 0.0, 0.0, 0.46826666666666666, 0.0, 0.04...",infeasible,infeasible,0.0,0.262291
4,4,4,4,30000309,87500,"[1400, 1445]","[1405, 1450]","[5, 5]","[1.688, 1.698]","[40.0, 570.0]",7,"['Public', 'Home']","['Home', 'Public', 'Home']","['""\'Home\'""', ' "" \'Public\'""', ' "" \'Home\'""']",1,1,3,2,26114,124754,1,0.036816,0.007882,0,0,"[Home, Public, Home, Public, Home, Public, Home]","[1.688, 1.698, 1.688, 1.698, 1.688, 1.698]","[40.0, 570.0, 40.0, 570.0, 40.0, 570.0]","[6, 50, 6, 50, 6, 50, 6]","[0.078, 0.35, 0.078, 0.35, 0.078, 0.35, 0.078]","[1.0, 0.0009575757575757651, 0.0, 0.0, 0.0, 0....","[1.0, 0.0, 0.0, 0.0, 0.0, 0.3645939393939394, ...","[-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.2433818181818182, ..."

In [58]:
eincome0 = df[df['income_classes'] == 0]
eincome1 = df[df['income_classes'] == 1]
eincome2 = df[df['income_classes'] == 2]

In [59]:
eincome0S3E2 = eincome0[eincome0['resultsS3E2']!='infeasible']
energylistS3E2_I0 = np.mean(eincome0S3E2['energyS3E2'])

    
eincome0S3E8 = eincome0[eincome0['resultsS3E8']!='infeasible']
energylistS3E8_I0 = np.mean(eincome0S3E8['energyS3E8'])


eincome0S5E2 = eincome0[eincome0['resultsS5E2']!='infeasible']
energylistS5E2_I0 = np.mean(eincome0S5E2['energyS5E2'])

     
eincome0S5E8 = eincome0[eincome0['resultsS5E8']!='infeasible']
energylistS5E8_I0 = np.mean(eincome0S5E8['energyS5E8'])



In [60]:
eincome1S3E2 = eincome1[eincome1['resultsS3E2']!='infeasible']
energylistS3E2_I1 = np.mean(eincome1S3E2['energyS3E2'])

    
eincome1S3E8 = eincome1[eincome1['resultsS3E8']!='infeasible']
energylistS3E8_I1 = np.mean(eincome1S3E8['energyS3E8'])


eincome1S5E2 = eincome1[eincome1['resultsS5E2']!='infeasible']
energylistS5E2_I1 = np.mean(eincome1S5E2['energyS5E2'])

     
eincome1S5E8 = eincome1[eincome1['resultsS5E8']!='infeasible']
energylistS5E8_I1 = np.mean(eincome1S5E8['energyS5E8'])




In [61]:
eincome2S3E2 = eincome2[eincome2['resultsS3E2']!='infeasible']
energylistS3E2_I2 = np.mean(eincome2S3E2['energyS3E2'])

    
eincome2S3E8 = eincome2[eincome2['resultsS3E8']!='infeasible']
energylistS3E8_I2 = np.mean(eincome2S3E8['energyS3E8'])


eincome2S5E2 = eincome2[eincome2['resultsS5E2']!='infeasible']
energylistS5E2_I2 = np.mean(eincome2S5E2['energyS5E2'])

     
eincome2S5E8 = eincome2[eincome2['resultsS5E8']!='infeasible']
energylistS5E8_I2 = np.mean(eincome2S5E8['energyS5E8'])



In [62]:

MUD.loc[1,'Income 1 (I30E20)'] = energylistS3E2_I0
MUD.loc[1,'Income 2 (I30E20)'] = energylistS3E2_I1
MUD.loc[1,'Income 3 (I30E20)'] = energylistS3E2_I2

MUD.loc[1,'Income 1 (I30E80)'] = energylistS3E8_I0
MUD.loc[1,'Income 2 (I30E80)'] = energylistS3E8_I1
MUD.loc[1,'Income 3 (I30E80)'] = energylistS3E8_I2

MUD.loc[1,'Income 1 (I50E20)'] = energylistS5E2_I0
MUD.loc[1,'Income 2 (I50E20)'] = energylistS5E2_I1
MUD.loc[1,'Income 3 (I50E20)'] = energylistS5E2_I2

MUD.loc[1,'Income 1 (I50E80)'] = energylistS5E8_I0
MUD.loc[1,'Income 2 (I50E80)'] = energylistS5E8_I1
MUD.loc[1,'Income 3 (I50E80)'] = energylistS5E8_I2

In [63]:
MUD

,ANALYSIS,Cluster 1 (I30E20),Cluster 2 (I30E20),Cluster 3 (I30E20),Cluster 1 (I30E80),Cluster 2 (I30E80),Cluster 3 (I30E80),Cluster 1 (I50E20),Cluster 2 (I50E20),Cluster 3 (I50E20),Cluster 1 (I50E80),Cluster 2 (I50E80),Cluster 3 (I50E80),Income 1 (I30E20),Income 2 (I30E20),Income 3 (I30E20),Income 1 (I30E80),Income 2 (I30E80),Income 3 (I30E80),Income 1 (I50E20),Income 2 (I50E20),Income 3 (I50E20),Income 1 (I50E80),Income 2 (I50E80),Income 3 (I50E80)
0,MUD share,0.766694,0.696864,0.832773,0.989085,1.000000,0.945201,0.435525,0.383117,0.596330,0.991717,1.00000,0.959055,0.757468,0.832507,0.928144,0.968459,0.954586,0.962963,0.442889,0.587696,0.734783,0.976802,0.966474,0.964286
1,TOTAL energy use,1.491591,0.411358,4.391943,3.898506,1.494902,9.473620,1.058319,0.477357,3.288343,4.104584,2.06326,9.042877,2.048228,3.946886,10.455053,4.829337,8.801246,16.562038,1.502802,2.926700,8.040426,4.870569,8.448707,14.458707
2,MUD energy use,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.674545,1.818539,4.320842,2.867338,5.547870,8.971941,0.508695,1.352440,3.424649,3.356782,5.911868,8.752762


In [71]:
ecluster0 = df[df['Cluster_new_features'] == 0]
ecluster1 = df[df['Cluster_new_features'] == 1]
ecluster2 = df[df['Cluster_new_features'] == 2]

In [72]:
ecluster0S3E2 = ecluster0[ecluster0['resultsS3E2']!='infeasible']
energylistS3E2_C0 = np.mean(ecluster0S3E2['energyS3E2'])

    
ecluster0S3E8 = ecluster0[ecluster0['resultsS3E8']!='infeasible']
energylistS3E8_C0 = np.mean(ecluster0S3E8['energyS3E8'])

    
ecluster0S5E2 = ecluster0[ecluster0['resultsS5E2']!='infeasible']
energylistS5E2_C0 = np.mean(ecluster0S5E2['energyS5E2'])
    
    
ecluster0S5E8 = ecluster0[ecluster0['resultsS5E8']!='infeasible']
energylistS5E8_C0 = np.mean(ecluster0S5E8['energyS5E8'])
    
    

    
ecluster1S3E2 = ecluster1[ecluster1['resultsS3E2']!='infeasible']
energylistS3E2_C1 = np.mean(ecluster1S3E2['energyS3E2'])
    
    
ecluster1S3E8 = ecluster1[ecluster1['resultsS3E8']!='infeasible']
energylistS3E8_C1 = np.mean(ecluster1S3E8['energyS3E8'])
    
    
ecluster1S5E2 = ecluster1[ecluster1['resultsS5E2']!='infeasible']
energylistS5E2_C1 = np.mean(ecluster1S5E2['energyS5E2'])
    
    
ecluster1S5E8 = ecluster1[ecluster1['resultsS5E8']!='infeasible']
energylistS5E8_C1 = np.mean(ecluster1S5E8['energyS5E8'])
    
    


    
ecluster2S3E2 = ecluster2[ecluster2['resultsS3E2']!='infeasible']
energylistS3E2_C2 = np.mean(ecluster2S3E2['energyS3E2'])
    
    
ecluster2S3E8 = ecluster2[ecluster2['resultsS3E8']!='infeasible']
energylistS3E8_C2 = np.mean(ecluster2S3E8['energyS3E8'])
    
    
ecluster2S5E2 = ecluster2[ecluster2['resultsS5E2']!='infeasible']
energylistS5E2_C2 = np.mean(ecluster2S5E2['energyS5E2'])
    
    
ecluster2S5E8 = ecluster2[ecluster2['resultsS5E8']!='infeasible']
energylistS5E8_C2 = np.mean(ecluster2S5E8['energyS5E8'])
    
    


In [73]:

MUD.loc[2,'Cluster 1 (I30E20)'] = energylistS3E2_C0
MUD.loc[2,'Cluster 2 (I30E20)'] = energylistS3E2_C1
MUD.loc[2,'Cluster 3 (I30E20)'] = energylistS3E2_C2

MUD.loc[2,'Cluster 1 (I30E80)'] = energylistS3E8_C0
MUD.loc[2,'Cluster 2 (I30E80)'] = energylistS3E8_C1
MUD.loc[2,'Cluster 3 (I30E80)'] = energylistS3E8_C2

MUD.loc[2,'Cluster 1 (I50E20)'] = energylistS5E2_C0
MUD.loc[2,'Cluster 2 (I50E20)'] = energylistS5E2_C1
MUD.loc[2,'Cluster 3 (I50E20)'] = energylistS5E2_C2

MUD.loc[2,'Cluster 1 (I50E80)'] = energylistS5E8_C0
MUD.loc[2,'Cluster 2 (I50E80)'] = energylistS5E8_C1
MUD.loc[2,'Cluster 3 (I50E80)'] = energylistS5E8_C2

In [74]:
MUD

,ANALYSIS,Cluster 1 (I30E20),Cluster 2 (I30E20),Cluster 3 (I30E20),Cluster 1 (I30E80),Cluster 2 (I30E80),Cluster 3 (I30E80),Cluster 1 (I50E20),Cluster 2 (I50E20),Cluster 3 (I50E20),Cluster 1 (I50E80),Cluster 2 (I50E80),Cluster 3 (I50E80),Income 1 (I30E20),Income 2 (I30E20),Income 3 (I30E20),Income 1 (I30E80),Income 2 (I30E80),Income 3 (I30E80),Income 1 (I50E20),Income 2 (I50E20),Income 3 (I50E20),Income 1 (I50E80),Income 2 (I50E80),Income 3 (I50E80)
0,MUD share,0.766694,0.696864,0.832773,0.989085,1.000000,0.945201,0.435525,0.383117,0.596330,0.991717,1.000000,0.959055,0.757468,0.832507,0.928144,0.968459,0.954586,0.962963,0.442889,0.587696,0.734783,0.976802,0.966474,0.964286
1,TOTAL energy use,1.491591,0.411358,4.391943,3.898506,1.494902,9.473620,1.058319,0.477357,3.288343,4.104584,2.063260,9.042877,2.048228,3.946886,10.455053,4.829337,8.801246,16.562038,1.502802,2.926700,8.040426,4.870569,8.448707,14.458707
2,MUD energy use,0.533160,0.194838,1.944168,2.503329,1.042210,5.823779,0.392058,0.192886,1.464326,3.049636,1.471668,6.191384,0.674545,1.818539,4.320842,2.867338,5.547870,8.971941,0.508695,1.352440,3.424649,3.356782,5.911868,8.752762


In [75]:
MUD.to_excel('AGGRegion_3days.xlsx')

In [94]:
energyCluster.to_excel("MUDEnergyCluster_W033P035_3days.xlsx",
...              sheet_name='Energy_Cluster')  
energyIncome.to_excel("MUDEnergyIncome_W033P035_3days.xlsx",
...              sheet_name='Energy_Income')  

In [ ]:
MudShare

In [ ]:

plt.style.use('fivethirtyeight')

x1 = MudShare['Cluster 1 (I30E20)']*100
x2 = MudShare['Cluster 2 (I30E20)']*100
x3 = MudShare['Cluster 3 (I30E20)']*100
x4 = MudShare['Cluster 1 (I30E80)']*100
x5 = MudShare['Cluster 2 (I30E80)']*100
x6 = MudShare['Cluster 3 (I30E80)']*100

y = MudShare['Census Division']
fig = plt.figure(figsize=(10, 8),dpi=300)
# fig(figsize=(8, 6), dpi=80)
ax1 = fig.add_subplot(111)

ax1.scatter(x1, y, s=50, c='b', marker="^", label='C1&S1')
ax1.scatter(x2, y, s=50, c='r', marker="^", label='C2&S1')
ax1.scatter(x3, y, s=50, c='g', marker="^", label='C3&S1')
ax1.scatter(x4, y, s=50, c='b', marker="o", label='C1&S2')
ax1.scatter(x5, y, s=50, c='r', marker="o", label='C2&S2')
ax1.scatter(x6, y, s=50, c='g', marker="o", label='C3&S2')
plt.xlabel("Share (%)")
plt.title('Expected share of MUDs deployment (starting SOC: 30 miles)')
plt.legend(loc='best',prop={'size': 7.2});
# plt.xticks(rotation=45,fontsize=14)
# plt.yticks(fontsize=14)
# plt.xlabel('Time of Day',fontsize=18)
# plt.ylabel('VMT (Miles)',fontsize=18)
# plt.title('MUDs residents VMT during time of day',fontsize=28)
plt.legend(fontsize=18)
plt.show()

fig.savefig('figure8aDiff.png',dpi=300)

In [ ]:
x1 = MudShare['Cluster 1 (I50E20)']*100
x2 = MudShare['Cluster 2 (I50E20)']*100
x3 = MudShare['Cluster 3 (I50E20)']*100
x4 = MudShare['Cluster 1 (I50E80)']*100
x5 = MudShare['Cluster 2 (I50E80)']*100
x6 = MudShare['Cluster 3 (I50E80)']*100

y = MudShare['Census Division']
fig = plt.figure(figsize=(10, 8),dpi=300)
# fig(figsize=(8, 6), dpi=80)
ax1 = fig.add_subplot(111)

ax1.scatter(x1, y, s=50, c='b', marker="^", label='C1&S1')
ax1.scatter(x2, y, s=50, c='r', marker="^", label='C2&S1')
ax1.scatter(x3, y, s=50, c='g', marker="^", label='C3&S1')
ax1.scatter(x4, y, s=50, c='b', marker="o", label='C1&S2')
ax1.scatter(x5, y, s=50, c='r', marker="o", label='C2&S2')
ax1.scatter(x6, y, s=50, c='g', marker="o", label='C3&S2')
plt.xlabel('Share (%)')
plt.title('Expected share of MUDs deployment (starting SOC: 50 miles)')
plt.legend(loc='upper center');
plt.legend(fontsize=18)
plt.show()

fig.savefig('figure8bDiff.png',dpi=300)

In [ ]:
MudShareIncome

In [ ]:
x1 = MudShareIncome['Income 1 (I30E20)']*100
x2 = MudShareIncome['Income 2 (I30E20)']*100
x3 = MudShareIncome['Income 3 (I30E20)']*100
x4 = MudShareIncome['Income 1 (I30E80)']*100
x5 = MudShareIncome['Income 2 (I30E80)']*100
x6 = MudShareIncome['Income 3 (I30E80)']*100

y = MudShareIncome['Census Division']
fig = plt.figure(figsize=(10, 8),dpi=300)
# fig(figsize=(8, 6), dpi=80)
ax1 = fig.add_subplot(111)

ax1.scatter(x1, y, s=50, c='b', marker="^", label='LI&S1')
ax1.scatter(x2, y, s=50, c='r', marker="^", label='MI&S1')
ax1.scatter(x3, y, s=50, c='g', marker="^", label='UI&S1')
ax1.scatter(x4, y, s=50, c='b', marker="o", label='LI&S2')
ax1.scatter(x5, y, s=50, c='r', marker="o", label='MI&S2')
ax1.scatter(x6, y, s=50, c='g', marker="o", label='UI&S2')
plt.xlabel("Share (%)")
plt.title('Expected share of MUDs deployment (starting SOC: 30 mi)')
plt.legend(loc='best',prop={'size': 7.5});
plt.legend(fontsize=18)
plt.show()

fig.savefig('figure8cDiff.png',dpi=300)

In [ ]:
x1 = MudShareIncome['Income 1 (I50E20)']*100
x2 = MudShareIncome['Income 2 (I50E20)']*100
x3 = MudShareIncome['Income 3 (I50E20)']*100
x4 = MudShareIncome['Income 1 (I50E80)']*100
x5 = MudShareIncome['Income 2 (I50E80)']*100
x6 = MudShareIncome['Income 3 (I50E80)']*100

y = MudShareIncome['Census Division']
fig = plt.figure(figsize=(10, 8),dpi=300)
# fig(figsize=(8, 6), dpi=80)
ax1 = fig.add_subplot(111)

ax1.scatter(x1, y, s=50, c='b', marker="^", label='LI&S1')
ax1.scatter(x2, y, s=50, c='r', marker="^", label='MI&S1')
ax1.scatter(x3, y, s=50, c='g', marker="^", label='UI&S1')
ax1.scatter(x4, y, s=50, c='b', marker="o", label='LI&S2')
ax1.scatter(x5, y, s=50, c='r', marker="o", label='MI&S2')
ax1.scatter(x6, y, s=50, c='g', marker="o", label='UI&S2')
plt.xlabel('Share (%)')
plt.title('Expected share of MUDs deployment (starting SOC: 50 mi)')
plt.legend(loc='best');
plt.legend(fontsize=18)
plt.show()

fig.savefig('figure8dDiff.png',dpi=300)

In [ ]:
energyCluster = pd.read_excel('EnergyCluster.xlsx')
energyCluster

In [ ]:
plt.style.use('fivethirtyeight')

x1 = energyCluster['Cluster 1 (I30E20)']*1
x2 = energyCluster['Cluster 2 (I30E20)']*1
x3 = energyCluster['Cluster 3 (I30E20)']*1
x4 = energyCluster['Cluster 1 (I30E80)']*1
x5 = energyCluster['Cluster 2 (I30E80)']*1
x6 = energyCluster['Cluster 3 (I30E80)']*1

y = energyCluster['Census Division']
fig = plt.figure(figsize=(10, 8), dpi=300)
# fig(figsize=(8, 6), dpi=80)
ax1 = fig.add_subplot(111)

ax1.scatter(x1, y, s=50, c='b', marker="^", label='C1&S1')
ax1.scatter(x2, y, s=50, c='r', marker="^", label='C2&S1')
ax1.scatter(x3, y, s=50, c='g', marker="^", label='C3&S1')
ax1.scatter(x4, y, s=50, c='b', marker="o", label='C1&S2')
ax1.scatter(x5, y, s=50, c='r', marker="o", label='C2&S2')
ax1.scatter(x6, y, s=50, c='g', marker="o", label='C3&S2')
plt.xlabel("Electricity & Energy (kWh)")
# plt.ylabel("Y axis label")
plt.title('Per vehicle energy use daily (starting SOC: 30 miles)')
plt.legend(loc='best',prop={'size': 7.5});
plt.legend(fontsize=18)
plt.show()

fig.savefig('figure9aDiffTotal.png',dpi=300)

In [ ]:
x1 = energyCluster['Cluster 1 (I50E20)']*1
x2 = energyCluster['Cluster 2 (I50E20)']*1
x3 = energyCluster['Cluster 3 (I50E20)']*1
x4 = energyCluster['Cluster 1 (I50E80)']*1
x5 = energyCluster['Cluster 2 (I50E80)']*1
x6 = energyCluster['Cluster 3 (I50E80)']*1

y = energyCluster['Census Division']
fig = plt.figure(figsize=(10, 8), dpi=300)
# fig(figsize=(8, 6), dpi=80)
ax1 = fig.add_subplot(111)

ax1.scatter(x1, y, s=50, c='b', marker="^", label='C1&S1')
ax1.scatter(x2, y, s=50, c='r', marker="^", label='C2&S1')
ax1.scatter(x3, y, s=50, c='g', marker="^", label='C3&S1')
ax1.scatter(x4, y, s=50, c='b', marker="o", label='C1&S2')
ax1.scatter(x5, y, s=50, c='r', marker="o", label='C2&S2')
ax1.scatter(x6, y, s=50, c='g', marker="o", label='C3&S2')
plt.xlabel("Electricity & Energy (KWh)")
# plt.ylabel("Y axis label")
plt.title('Per vehicle energy use daily (starting SOC: 50 miles)')
plt.legend(loc='upper center',prop={'size': 7.5});
plt.legend(fontsize=18)
plt.show()

fig.savefig('figure9bDiffTotal.png',dpi=300)

In [ ]:
energyIncome = pd.read_excel('EnergyIncome.xlsx')
energyIncome

In [ ]:
x1 = energyIncome['Income 1 (I30E20)']*1
x2 = energyIncome['Income 2 (I30E20)']*1
x3 = energyIncome['Income 3 (I30E20)']*1
x4 = energyIncome['Income 1 (I30E80)']*1
x5 = energyIncome['Income 2 (I30E80)']*1
x6 = energyIncome['Income 3 (I30E80)']*1

y = energyIncome['Census Division']
fig = plt.figure(figsize=(10, 8), dpi=300)
# fig(figsize=(8, 6), dpi=80)
ax1 = fig.add_subplot(111)

ax1.scatter(x1, y, s=50, c='b', marker="^", label='LI&S1')
ax1.scatter(x2, y, s=50, c='r', marker="^", label='MI&S1')
ax1.scatter(x3, y, s=50, c='g', marker="^", label='UI&S1')
ax1.scatter(x4, y, s=50, c='b', marker="o", label='LI&S2')
ax1.scatter(x5, y, s=50, c='r', marker="o", label='MI&S2')
ax1.scatter(x6, y, s=50, c='g', marker="o", label='UI&S2')
plt.xlabel("Electricity & Energy (KWh)")
# plt.ylabel("Y axis label")
plt.title('Per vehicle energy use daily of income classes (starting SOC: 30 miles)')
plt.legend(loc='best',prop={'size': 7.5});
plt.legend(fontsize=18)
plt.show()

fig.savefig('figure9cDiffTotal.png',dpi=300)

In [ ]:
x1 = energyIncome['Income 1 (I50E20)']*1
x2 = energyIncome['Income 2 (I50E20)']*1
x3 = energyIncome['Income 3 (I50E20)']*1
x4 = energyIncome['Income 1 (I50E80)']*1
x5 = energyIncome['Income 2 (I50E80)']*1
x6 = energyIncome['Income 3 (I50E80)']*1

y = energyIncome['Census Division']
fig = plt.figure(figsize=(10, 8), dpi=300)
# fig(figsize=(8, 6), dpi=80)
ax1 = fig.add_subplot(111)

ax1.scatter(x1, y, s=50, c='b', marker="^", label='LI&S1')
ax1.scatter(x2, y, s=50, c='r', marker="^", label='MI&S1')
ax1.scatter(x3, y, s=50, c='g', marker="^", label='UI&S1')
ax1.scatter(x4, y, s=50, c='b', marker="o", label='LI&S2')
ax1.scatter(x5, y, s=50, c='r', marker="o", label='MI&S2')
ax1.scatter(x6, y, s=50, c='g', marker="o", label='UI&S2')
plt.xlabel("Electricity & Energy (KWh)")
# plt.ylabel("Y axis label")
plt.title('Per vehicle energy use daily of income classes (starting SOC: 50 miles)')
plt.legend(loc='upper right',prop={'size': 7.5});
plt.legend(fontsize=18)
plt.show()

fig.savefig('figure9dDiffTotal.png',dpi=300)